In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Read the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv( "weather_data/WeatherPy_vacation.csv")
city_data_df.head(60)


,City_ID,City_ID.1,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
0,0,0,Mahebourg,MU,2020-01-05 11:19:04,-20.41,57.70,81.00,83,75,8.05,0.00,0.00,broken clouds
1,1,1,Hobart,AU,2020-01-05 11:18:10,-42.88,147.33,60.80,55,20,11.41,0.00,0.00,few clouds
2,2,2,Yellowknife,CA,2020-01-05 11:15:11,62.45,-114.38,-2.00,84,90,13.87,0.00,0.00,light snow
3,3,3,Avarua,CK,2020-01-05 11:19:05,-21.21,-159.78,77.00,94,100,3.36,0.00,0.00,overcast clouds
4,4,4,Turukhansk,RU,2020-01-05 11:19:06,65.80,87.96,18.19,94,100,16.42,1.44,0.00,light snow
5,5,5,Juneau,US,2020-01-05 11:19:06,58.30,-134.42,27.00,85,75,4.70,0.00,0.00,broken clouds
6,6,6,Faya,SA,2020-01-05 11:19:06,18.39,42.45,68.00,59,40,14.99,0.00,0.00,scattered clouds
7,7,7,Mataura,NZ,2020-01-05 11:19:06,-46.19,168.86,50.00,77,64,11.01,0.00,0.31,light rain
8,8,8,Kavieng,PG,2020-01-05 11:19:07,-2.57,150.80,84.88,73,65,11.10,0.00,3.50,moderate rain
9,9,9,Ponta Do Sol,BR,2020-01-05 11:19:07,-20.63,-46.00,73.44,81,46,4.88,0.00,0.00,scattered clouds


In [3]:
# Filter the dataset to find the cities that fit the criteria.
zero = 0.0
preferred_cities_df = city_data_df.loc[(city_data_df["City_ID"] == 38) | 
                                       (city_data_df["City_ID"] == 40) |
                                       (city_data_df["City_ID"] == 51) |
                                       (city_data_df["City_ID"] == 45) ]
preferred_cities_df.head(100)

,City_ID,City_ID.1,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(last 3hrs),Snow inches(last 3hrs),Current Description
38,38,38,Cupertino,US,2020-01-05 11:14:27,37.32,-122.03,50.00,93,20,6.93,0.0,0.0,light rain
40,40,40,Port Elizabeth,US,2020-01-05 11:19:15,39.31,-74.98,41.00,55,90,17.22,0.0,0.0,overcast clouds
45,45,45,Sitka,US,2020-01-05 11:19:17,37.17,-99.65,44.01,53,28,18.99,0.0,0.0,scattered clouds
51,51,51,Craig,US,2020-01-05 11:19:18,40.52,-107.55,19.40,77,1,3.36,0.0,0.0,clear sky


In [4]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Description"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
38,Cupertino,US,50.00,37.32,-122.03,light rain,
40,Port Elizabeth,US,41.00,39.31,-74.98,overcast clouds,
45,Sitka,US,44.01,37.17,-99.65,scattered clouds,
51,Craig,US,19.40,40.52,-107.55,clear sky,


In [6]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()  
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping."+row["City"])

Hotel not found... skipping.Sitka


In [7]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
#locations[["Lat", "Lng"][0]]
locations

,Lat,Lng
38,37.32,-122.03
40,39.31,-74.98
45,37.17,-99.65
51,40.52,-107.55


In [26]:
max_temp = [max(temp, 0) for temp in hotel_df["Max Temp"]]
# max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# geneva2zurich = gmaps.directions_layer(geneva, zurich)
#direction gives me an error
#direction =  gmaps.directions_layer(locations, 
#   stroke_color='red', stroke_weight=3.0, travel_mode='DRIVING', stroke_opacity=1.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
#fig.add_layer(direction)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))